In [ ]:
import pandas as pd
import utils_max_flow as utils
import pickle

In [ ]:
with open('graph_objects/G_simple_directed_iggielgn.pickle', 'rb') as f:
    G_simple_directed = pickle.load(f)
    G_simple_directed.name = 'G_simple_directed'

In [ ]:
# Dataset; 'iggin' or 'iggielgn'
dataset = 'iggielgn'

# S-t-flow; 's-t-degree' or 's-t-type'
s_t_flow = 'hybrid'

In [ ]:
# Toggle to run N-k or just print stored results
run_analysis = True

----
# Heuristics

### Node removal

In [ ]:
if run_analysis:
    random_node_removal_df = utils.flow_capacity_robustness(G_simple_directed, heuristic='random', remove='node', k_removals=2500, n_benchmarks=5)
    random_node_removal_df.to_pickle('results/max_flow/'+dataset+'/'+s_t_flow+'/random_node_removal_df.pkl')
random_node_removal_df = pd.read_pickle('results/max_flow/'+dataset+'/'+s_t_flow+'/random_node_removal_df.pkl')

In [ ]:
if run_analysis:
    load_rate_node_removal_df = utils.flow_capacity_robustness(G_simple_directed, heuristic='load_rate', remove='node')
    load_rate_node_removal_df.to_pickle('results/max_flow/'+dataset+'/'+s_t_flow+'/load_rate_node_removal_df.pkl')
load_rate_node_removal_df = pd.read_pickle('results/max_flow/'+dataset+'/'+s_t_flow+'/load_rate_node_removal_df.pkl')
utils.results_summary(load_rate_node_removal_df)

In [ ]:
if run_analysis:
    max_flow_node_removal_df = utils.flow_capacity_robustness(G_simple_directed, heuristic='max_flow', remove='node')
    max_flow_node_removal_df.to_pickle('results/max_flow/'+dataset+'/'+s_t_flow+'/max_flow_node_removal_df.pkl')
max_flow_node_removal_df = pd.read_pickle('results/max_flow/'+dataset+'/'+s_t_flow+'/max_flow_node_removal_df.pkl')
utils.results_summary(max_flow_node_removal_df)

In [ ]:
if run_analysis:
    max_flow_edge_count_node_removal_df = utils.flow_capacity_robustness(G_simple_directed, heuristic='max_flow_edge_count', remove='node')
    max_flow_edge_count_node_removal_df.to_pickle('results/max_flow/'+dataset+'/'+s_t_flow+'/max_flow_edge_count_node_removal_df.pkl')
max_flow_edge_count_node_removal_df = pd.read_pickle('results/max_flow/'+dataset+'/'+s_t_flow+'/max_flow_edge_count_node_removal_df.pkl')
utils.results_summary(max_flow_edge_count_node_removal_df)

In [ ]:
if run_analysis:
    wfcr_node_removal_df = utils.flow_capacity_robustness(G_simple_directed, heuristic='wfcr', remove='node')
    wfcr_node_removal_df.to_pickle('results/max_flow/'+dataset+'/'+s_t_flow+'/wfcr_node_removal_df.pkl')
wfcr_node_removal_df = pd.read_pickle('results/max_flow/'+dataset+'/'+s_t_flow+'/wfcr_node_removal_df.pkl')
utils.results_summary(wfcr_node_removal_df, 'capacity_robustness_max_flow')

In [ ]:
fig = utils.plot_heuristic_comparison_biplot([load_rate_node_removal_df, max_flow_edge_count_node_removal_df, max_flow_node_removal_df, wfcr_node_removal_df])
fig.savefig('saved_plots/'+dataset+'/'+s_t_flow+'/max_flow/node_removals.png')

### Edge removal

In [ ]:
if run_analysis:
    random_edge_removal_df = utils.flow_capacity_robustness(G_simple_directed, heuristic='random', remove='edge', k_removals=2500, n_benchmarks=5)
    random_edge_removal_df.to_pickle('results/max_flow/'+dataset+'/'+s_t_flow+'/random_edge_removal_df.pkl')
random_edge_removal_df = pd.read_pickle('results/max_flow/'+dataset+'/'+s_t_flow+'/random_edge_removal_df.pkl')

In [ ]:
if run_analysis:
    load_rate_edge_removal_df = utils.flow_capacity_robustness(G_simple_directed, heuristic='load_rate', remove='edge')
    load_rate_edge_removal_df.to_pickle('results/max_flow/'+dataset+'/'+s_t_flow+'/load_rate_edge_removal_df.pkl')
load_rate_edge_removal_df = pd.read_pickle('results/max_flow/'+dataset+'/'+s_t_flow+'/load_rate_edge_removal_df.pkl')
utils.results_summary(load_rate_edge_removal_df) 

In [ ]:
if run_analysis:
    max_flow_edge_removal_df = utils.flow_capacity_robustness(G_simple_directed, heuristic='max_flow', remove='edge')
    max_flow_edge_removal_df.to_pickle('results/max_flow/'+dataset+'/'+s_t_flow+'/max_flow_edge_removal_df.pkl')
max_flow_edge_removal_df = pd.read_pickle('results/max_flow/'+dataset+'/'+s_t_flow+'/max_flow_edge_removal_df.pkl')
utils.results_summary(max_flow_edge_removal_df)

In [ ]:
if run_analysis:
    max_flow_edge_count_edge_removal_df = utils.flow_capacity_robustness(G_simple_directed, heuristic='max_flow_edge_count', remove='edge')
    max_flow_edge_count_edge_removal_df.to_pickle('results/max_flow/'+dataset+'/'+s_t_flow+'/max_flow_edge_count_edge_removal_df.pkl')
max_flow_edge_count_edge_removal_df = pd.read_pickle('results/max_flow/'+dataset+'/'+s_t_flow+'/max_flow_edge_count_edge_removal_df.pkl')
utils.results_summary(max_flow_edge_count_edge_removal_df)

In [ ]:
if run_analysis:
    wfcr_edge_removal_df = utils.flow_capacity_robustness(G_simple_directed, heuristic='wfcr', remove='edge')
    wfcr_edge_removal_df.to_pickle('results/max_flow/'+dataset+'/'+s_t_flow+'/wfcr_edge_removal_df.pkl')
wfcr_edge_removal_df = pd.read_pickle('results/max_flow/'+dataset+'/'+s_t_flow+'/wfcr_edge_removal_df.pkl')
utils.results_summary(wfcr_edge_removal_df, 'capacity_robustness_max_flow') 

In [ ]:
fig = utils.plot_heuristic_comparison_biplot([load_rate_edge_removal_df, max_flow_edge_removal_df, max_flow_edge_count_edge_removal_df, wfcr_edge_removal_df])
fig.savefig('saved_plots/'+dataset+'/'+s_t_flow+'/max_flow/edge_removals.png')